In [1]:
# %pip install scikit-surprise==1.1.0 # pickle

In [2]:
from surprise import accuracy, Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [4]:
import pandas as pd
import pathlib
DATA_DIR = pathlib.Path().resolve().parent / "data"
print(DATA_DIR)
DATA_DIR.exists()

/Users/cfe/Dev/recommender/src/data


True

In [5]:
dataset = DATA_DIR / 'ratings_small.csv'
dataset.exists()

True

In [6]:
df = pd.read_csv(dataset)
df['rating'].dropna(inplace=True)
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
df.rating.max(), df.rating.min()

(5.0, 0.5)

In [8]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [9]:
algo = SVD(verbose=True, n_epochs=20)
cross_validate(algo, data, measures=['RMSE', "MAE"], cv=4, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing

{'test_rmse': array([0.89634768, 0.90290039, 0.90201402, 0.89608067]),
 'test_mae': array([0.69166537, 0.6945926 , 0.69311824, 0.69098702]),
 'fit_time': (4.50624418258667,
  4.4077160358428955,
  4.4726951122283936,
  4.415704011917114),
 'test_time': (0.1751549243927002,
  0.15494704246520996,
  0.16430377960205078,
  0.15254878997802734)}

In [10]:
trainset = data.build_full_trainset()
algo.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [13]:
testset = trainset.build_testset()
predictions = algo.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)
# accuracy.mae(predictions, verbose=True)

RMSE: 0.6427


0.6426728408673613

In [46]:
sample_row = df.sample(n=1)
userId = sample_row['userId'].values[0]
movieId = sample_row['movieId'].values[0]
print(userId, movieId)

646 3


In [47]:
pred = algo.predict(uid=userId, iid=movieId)
pred.est

4.176235080161417

In [41]:
import pickle

In [42]:
algo_data = {"model": algo}
with open('model.pkl', 'wb') as f:
    pickle.dump(algo_data, f)

In [43]:
model_algo= None
with open('model.pkl', 'rb') as f:
    model_data_loaded = pickle.load(f)
    model_algo = model_data_loaded.get('model')

In [48]:
model_algo.predict(uid=userId, iid=movieId).est

4.176235080161417

In [53]:
sample_rows = df.sample(n=10).to_dict('records')
for row in sample_rows:
    userId = row['userId']
    movieId = row['movieId']
    pred = model_algo.predict(uid=userId, iid=movieId).est
    print(userId, movieId, pred)

380 1517 3.8627797673686977
295 480 4.300739005098922
602 144 2.769952225952014
427 5902 3.9085105108428366
110 374 3.4816203628630493
559 1962 4.431293011796753
56 142488 3.7747479340719323
13 2762 3.631353002001287
168 410 2.891752026250828
311 1009 2.841714863690881
